In [1]:
#Importacion de las librerias que se van a utilizar.
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
# Cargo el archivo CSV en un DataFrame para poder utilizarlo.
df = pd.read_csv('plataformas.csv',parse_dates=['date_added'])

<b>Crear 7 funciones para la FastApi:<b>



Película (sólo película, no serie, ni documentales, etc) con mayor duración según año, plataforma y tipo de duración. La función debe llamarse get_max_duration(year, platform, duration_type) y debe devolver sólo el string del nombre de la película.


In [3]:
def get_max_duration(anio: int, plataforma: str, dtype: str):  
    
    # Me aseguro que si el usuario escribe en mayus se pase a minusculas.
    dtype = dtype.lower()
    platform = plataforma.lower()[0] #Aclaro indice en 0 para que tome la primer letra

    # filtro por año de lanzamiento y plataforma aclarando que solo tenga en cuenta la primer letra de la columna ID.   
    data_filtrada = df[(df['release_year'] == anio) & (df['id'].str.startswith(platform)) & (df['duration_type'] == dtype) & (df['type'] == 'movie')]    
    
    max_duration = data_filtrada['duration_int'].max()
    max_duration_row = data_filtrada.loc[data_filtrada['duration_int'] == max_duration]    
       
    if not max_duration_row.empty:
        # obtener el nombre de la película y devolverlo como una cadena de texto
        movie_name = max_duration_row['title'].iloc[0]
        return {'pelicula': movie_name}
    else:
        return {'No se encontraron peliculas que cumplan con los criterios de busqueda.'}

In [4]:
get_max_duration(2020,'NETFLIX','min')

{'pelicula': 'unbreakable kimmy schmidt: kimmy vs. the reverend'}

In [5]:
get_max_duration(2018,'amazon','min')


{'pelicula': 'soothing surf at del norte for sleep black screen'}

In [6]:
get_max_duration(2020,'disney','min')


{'pelicula': "dory's reef cam"}

In [7]:
get_max_duration(2020,'hulu','min')

{'pelicula': 'crock of gold: a few rounds with shane macgowan'}

--------------------------------------------------------------------------------------------------------------------------------------------------------


Cantidad de películas (sólo películas, no series, ni documentales, etc) según plataforma, con un puntaje mayor a XX en determinado año. La función debe llamarse get_score_count(platform, scored, year) y debe devolver un int, con el total de películas que cumplen lo solicitado.


In [8]:
def get_score_count(plataforma: str, scored: float, year: int):    
    # Me aseguro que si el usuario escribe en mayus se pase a minusculas, en el caso de plataform solo me interesa la primer letra.
    platform = plataforma.lower()[0]#Aclaro indice en 0 para que tome la primer letra
        

    data_filtrada = df[(df['id'].str.startswith(platform)) & (df['release_year'] == year) & (df['type'] == 'movie')]
    
    # contar la cantidad de películas que tienen el rating deseado
    count = data_filtrada[data_filtrada['score'] > scored]['id'].count()
    
    return {'plataforma': plataforma,
            'cantidad': int(count),
            'anio': year,
            'score': scored}

In [9]:
get_score_count('NetFlix',3.5, 2020)


{'plataforma': 'NetFlix', 'cantidad': 190, 'anio': 2020, 'score': 3.5}

In [10]:
get_score_count('amazon',3.5, 2020)


{'plataforma': 'amazon', 'cantidad': 267, 'anio': 2020, 'score': 3.5}

In [11]:
get_score_count('hulu',3.5, 2020)


{'plataforma': 'hulu', 'cantidad': 83, 'anio': 2020, 'score': 3.5}

In [12]:
get_score_count('disney',3.5, 2020)

{'plataforma': 'disney', 'cantidad': 34, 'anio': 2020, 'score': 3.5}

--------------------------------------------------------------------------------------------------------------------------------------------------------


Cantidad de películas (sólo películas, no series, ni documentales, etc) según plataforma. La función debe llamarse get_count_platform(platform) y debe devolver un int, con el número total de películas de esa plataforma. Las plataformas deben llamarse amazon, netflix, hulu, disney.


In [13]:
def get_count_platform(plataforma: str):
        
    # Me aseguro que si el usuario escribe en mayus se pase a minusculas, en el caso de plataform solo me interesa la primer letra.
    platform = plataforma.lower()[0]#Aclaro indice en 0 para que tome la primer letra
    
    # Realizo el filtro para que cuente del dataframe solo la plataforma de la columna ID y movie de la columna type
    data_filtrada = df[(df['id'].str.startswith(platform)) & (df['type'] == 'movie')].shape[0]    

    return {'plataforma': plataforma,
            'peliculas': data_filtrada}

In [14]:
get_count_platform('hulu')


{'plataforma': 'hulu', 'peliculas': 1484}

In [15]:
get_count_platform('amazon')


{'plataforma': 'amazon', 'peliculas': 7814}

In [16]:
get_count_platform('disney')


{'plataforma': 'disney', 'peliculas': 1052}

In [17]:
get_count_platform('netflix')

{'plataforma': 'netflix', 'peliculas': 6131}

--------------------------------------------------------------------------------------------------------------------------------------------------------


Actor que más se repite según plataforma y año. La función debe llamarse get_actor(platform, year) y debe devolver sólo el string con el nombre del actor que más se repite según la plataforma y el año dado.


In [18]:
def get_actor(plataforma: str, anio: int):    
    
    # Me aseguro que si el usuario escribe en mayus se pase a minusculas, en el caso de plataform solo me interesa la primer letra.
    platform = plataforma.lower()[0]#Aclaro indice en 0 para que tome la primer letra
    
    
    data_filtrada = df[(df['id'].str.startswith(platform)) & (df['release_year'] == anio)]   
 
    # En este paso, se utiliza la función dropna() de pandas para eliminar los valores NaN de la columna "cast" del DataFrame. 
    # Luego, se utiliza el método apply() para aplicar la función str() a cada valor de la columna "cast", convirtiendo los valores a cadenas de caracteres. 
    # Finalmente, se utiliza el método str.split(',') para dividir cada cadena de caracteres en una lista de actores utilizando la coma como separador. 
    # El resultado es una serie de pandas que contiene una lista de actores para cada fila del DataFrame original.
    actores_por_fila = data_filtrada['cast'].dropna().apply(lambda x: [i.strip() for i in x.split(',') if not i.strip().isdigit()])


    # Cuento la cantidad de veces que aparece cada actor en todas las filas, utilizando la clase Counter de Python.
    contador_actores = Counter()
    for actores in actores_por_fila:
        contador_actores.update(actores)

    # Encuentro el actor que aparece más veces utilizando la funcion most common devolviendo una lista de tuplas donde cada tupla contiene un actor 
    # y la cantidad de veces que aparece en todas las filas del DataFrame.
    # se asigna [0][0] para indicar el actor que mas veces aparece
    actor_mas_repetido = contador_actores.most_common(1)
    
    if actor_mas_repetido:
        # se asigna [0][0] para indicar el actor que mas veces aparece
        actor_mas_repetido = actor_mas_repetido[0][0]
    else:
        return {'plataforma': plataforma,
                'anio': anio,
                'actor': "No hay datos disponibles",
                'apariciones': "No hay datos disponibles"}

    # Muestro el actor que aparece más veces y la cantidad de veces que aparece
    cantidad_actor_mas_repetido = contador_actores[actor_mas_repetido]
    return {'plataforma': plataforma,
            'anio': anio,
            'actor': actor_mas_repetido,
            'apariciones': cantidad_actor_mas_repetido}

In [19]:
get_actor('amazon',2021)


{'plataforma': 'amazon', 'anio': 2021, 'actor': 'om nom', 'apariciones': 10}

In [20]:
get_actor('hulu',2020)


{'plataforma': 'hulu',
 'anio': 2020,
 'actor': 'No hay datos disponibles',
 'apariciones': 'No hay datos disponibles'}

In [21]:
get_actor('netflix',2020)


{'plataforma': 'netflix',
 'anio': 2020,
 'actor': 'blossom chukwujekwu',
 'apariciones': 6}

In [22]:
get_actor('disney',2020)

{'plataforma': 'disney',
 'anio': 2020,
 'actor': 'daveed diggs',
 'apariciones': 3}

--------------------------------------------------------------------------------------------------------------------------------------------------------


La cantidad de contenidos/productos (todo lo disponible en streaming) que se publicó por país y año. La función debe llamarse prod_per_county(tipo,pais,anio) deberia devolver el tipo de contenido (pelicula,serie) por pais y año en un diccionario con las variables llamadas 'pais' (nombre del pais), 'anio' (año), 'pelicula' (tipo de contenido).


In [23]:
def prod_per_county(tipo: str, pais: str, anio: int):   
    
    data_filtrada = df[(df['country'] == pais) & (df['type'] == tipo) & (df['release_year'] == anio)]
    
    # contar el número de filas que cumplen con los criterios
    respuesta = len(data_filtrada)    
    
    return {'pais': pais,
            'anio': anio,            
            'peliculas': respuesta}

In [24]:
df['country'].value_counts()

united states                                                  4997
india                                                          1203
united kingdom                                                  583
japan                                                           515
canada                                                          237
                                                               ... 
lebanon, united arab emirates, france, switzerland, germany       1
france, belgium, italy                                            1
lebanon, united states, united arab emirates                      1
lebanon, france                                                   1
united arab emirates, jordan                                      1
Name: country, Length: 886, dtype: int64

In [25]:
def prod_per_county(tipo: str, pais: str, anio: int):   
    
    # Realizo un filtro para que busque por tipo, año de lanzamiento y que el campo "country" contenga el país.
    data_filtrada = df[(df['type'] == tipo) & (df['release_year'] == anio) & (df['country'].str.contains(pais))]
    
    # Separar los países en una lista y contarlos individualmente
    count_por_pais = {}
    for countries in data_filtrada['country']:
        for country in countries.split(','):
            if country.strip() not in count_por_pais:
                count_por_pais[country.strip()] = 0
            count_por_pais[country.strip()] += 1
    
    # Obtener el recuento de películas para el país especificado
    respuesta = count_por_pais.get(pais, 0)
    
    return {'pais': pais,
            'anio': anio,            
            'peliculas': respuesta}


In [26]:
prod_per_county('movie','united arab emirates' , 2014)

{'pais': 'united arab emirates', 'anio': 2014, 'peliculas': 5}

--------------------------------------------------------------------------------------------------------------------------------------------------------


La cantidad total de contenidos/productos (todo lo disponible en streaming, series, documentales, peliculas, etc) según el rating de audiencia dado (para que publico fue clasificada la pelicula). La función debe llamarse get_contents(rating) y debe devolver el numero total de contenido con ese rating de audiencias.

In [27]:
def get_contents(rating: str):       
    
    respuesta =  df['rating'].value_counts()[rating]

    return {'rating': rating,
            'contenido': int(respuesta)}

In [28]:
get_contents('g')

{'rating': 'g', 'contenido': 1269}

--------------------------------------------------------------------------------------------------------------------------------------------------------

<b>Version A<b>

In [29]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['title'])


In [30]:
def get_recommendations(movie_title: str):
    indices = pd.Series(df.index, index=df['title'])
    idx = indices[movie_title]

    cosine_sim = cosine_similarity(tfidf_matrix[idx], tfidf_matrix)
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    respuesta = df['title'].iloc[movie_indices].tolist()
    return {'recomendacion': respuesta}


In [31]:
get_recommendations('the grand seduction')

{'recomendacion': ['the grand',
  'five grand',
  'grand hotel',
  'the grand tour',
  'grand army']}

In [32]:
get_recommendations('tarzan')

{'recomendacion': ['tarzan',
  'tarzan',
  'tarzan 2',
  'tarzan ii',
  'come on, tarzan']}

<b>Version B<b>

In [33]:
def get_recommendations(title: str):
    vectorizer = TfidfVectorizer()
    # Aplicar el vectorizador a la columna 'listed_in' para obtener la matriz de características
    tfidf_matrix = vectorizer.fit_transform(df['listed_in'])
    # Calcular la matriz de similitud de coseno
    cosine_sim = cosine_similarity(tfidf_matrix)
    # Obtener el índice de la película que coincide con el título proporcionado
    idx = df.index[df['title'] == title.lower()].tolist()[0]

    # Obtener las puntuaciones de similitud de coseno de la película con todas las demás películas
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordenar las películas según las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener las cinco películas más similares, excluyendo la película de entrada
    sim_scores = [i for i in sim_scores if df.index[i[0]] != idx]
    sim_scores = sim_scores[:5]

    # Obtener los títulos de las cinco películas más similares
    recomendaciones = df.iloc[[i[0] for i in sim_scores]]['title'].tolist()

    # Devolver las recomendaciones en el formato especificado
    return {'recomendacion': recomendaciones}

In [34]:
get_recommendations('the grand seduction')

{'recomendacion': ["summer '03",
  'z: the beginning of everything',
  'you, me and him',
  'walter',
  'waiting on mary']}

In [35]:
get_recommendations('tarzan')

{'recomendacion': ['on happiness road',
  'invincible',
  'metal skin panic madox-1',
  'series before 1c onboarding - 2',
  'nuttiest nutcracker']}

VERSION C

In [36]:
def get_recommendations(title: str):
    df_corr = pd.read_csv('ML.csv')
    # Crear el vectorizador TF-IDF
    vectorizer = TfidfVectorizer()
    # Aplicar el vectorizador a la columna 'listed_in' para obtener la matriz de características
    tfidf_matrix = vectorizer.fit_transform(df_corr['title'])
    # Calcular la matriz de similitud de coseno
    cosine_sim = cosine_similarity(tfidf_matrix) 
    idx = df_corr.index[df_corr['title'] == title.lower()].tolist()[0]
    # Obtener las puntuaciones de similitud de coseno de la película con todas las demás películas
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Ordenar las películas según las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Obtener las cinco películas más similares, excluyendo la película de entrada
    sim_scores = [i for i in sim_scores if df_corr.index[i[0]] != idx]
    sim_scores = sim_scores[:5]
    # Obtener los títulos de las cinco películas más similares
    recomendaciones = df_corr.iloc[[i[0] for i in sim_scores]]['title'].tolist()
    # Devolver las recomendaciones en el formato especificado
    return {'recomendacion': recomendaciones}

In [37]:
get_recommendations('the grand seduction')

{'recomendacion': ['the grand',
  'five grand',
  'grand isle',
  'winged seduction: birds of paradise',
  'into the grand canyon']}

In [38]:
get_recommendations('tarzan')

{'recomendacion': ['tarzan',
  'tarzan',
  'tarzan 2',
  'tarzan ii',
  'tarzan & jane']}

VERSION FINAL

In [39]:
df_corr = pd.read_csv('ML.csv')

In [40]:
def get_recommendations(title: str):    
    # Crear el vectorizador TF-IDF
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df_corr['categorias'])
    
    # Obtener el índice de la película que coincide con el título proporcionado
    idx = df_corr.index[df_corr['title'] == title.lower()].tolist()[0]
    
    # Calcular la matriz de similitud de coseno
    cosine_sim = cosine_similarity(tfidf_matrix[idx], tfidf_matrix)
    
    # Obtener las puntuaciones de similitud de coseno de la película con todas las demás películas
    sim_scores = list(enumerate(cosine_sim[0]))
    
    # Ordenar las películas según las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obtener las cinco películas más similares, excluyendo la película de entrada
    sim_scores = [i for i in sim_scores if i[0] != idx]
    sim_scores = sim_scores[:5]
    
    # Obtener los títulos de las cinco películas más similares
    recomendaciones = df_corr.iloc[[i[0] for i in sim_scores]]['title'].tolist()
    
    # Devolver las recomendaciones en el formato especificado
    return {'recomendacion': recomendaciones} 

In [41]:
get_recommendations('the grand seduction')

{'recomendacion': ['take care good night',
  'grease live!',
  "david's mother",
  'take care',
  "summer '03"]}

In [42]:
get_recommendations('tarzan')

{'recomendacion': ['stay tuned',
  'technotise: edit & i',
  'on happiness road',
  'the congress',
  'shrek 2']}